## LOAD DATASET (PREPROCESSED DATA)

In [2]:
import tensorflow as tf
from tensorflow.keras.preprocessing import  image_dataset_from_directory
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # To show all logs, including debug messages


In [2]:
# import torch
# print("CUDA available:", torch.cuda.is_available())
# print("GPU name:", torch.cuda.get_device_name(0))

In [3]:
PROCESSED_DATASET_PATH = r"Dataset/ProcessedData"

# GETTING THE DATASET FROM PREPROCESSED DATASET

In [23]:
dataset = image_dataset_from_directory(
        PROCESSED_DATASET_PATH,
        image_size = (224,224),
        batch_size  = 5,       # it will take 32 images at ocne for processing
        labels = "inferred",     #name of the  class will be taken from the folder name
        label_mode = 'categorical',  #telling the model that output label will be integer type
        shuffle = True,
        seed = 42
       
)
            


Found 34166 files belonging to 12 classes.


In [24]:
len(dataset)

6834

In [17]:
train_size = int(0.8 * len(dataset))
test_size  = int(0.2 * len(dataset))



In [22]:
train_size

<class 'int'>


In [19]:
test_size

6833

## ASSIGNING THE DATASETS TO THE SPECIFIC VARIABLES 


splitting the dataset to train dataset test dataset and validation dataset

In [6]:
train_dataset = dataset.take(train_size)
test_dataset = dataset.take(test_size)


In [7]:
train_dataset

<_TakeDataset element_spec=(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>

In [8]:
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau


In [9]:

# Dynamically determine the number of classes
INPUT_SHAPE = (224, 224, 3)  # Input shape of the images
EPOCHS = 10
LEARNING_RATE = 0.1


In [10]:

# Step 1: Define the Model
def create_model():
    model = models.Sequential([
        # Base Convolutional Layers
        layers.Conv2D(filters = 64, kernel_size = (3, 3), activation='relu', input_shape=INPUT_SHAPE),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(filters = 128, kernel_size = (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
   
        

        # Flatten and Fully Connected Layers
        layers.Flatten(),
        # layers.GlobalAveragePooling2D(),
        layers.Dense(500, activation='relu',kernel_regularizer=tf.keras.regularizers.l2(0.01)),
        layers.Dense(256, activation='relu',kernel_regularizer=tf.keras.regularizers.l2(0.01)),
        layers.Dense(128, activation='relu',kernel_regularizer=tf.keras.regularizers.l2(0.01)),
        
        
        layers.Dense(12,activation='softmax')  # Final layer for classification
    ])
    return model


In [11]:

# Step 2: Compile the Model
model = create_model()

model.compile(
    optimizer=tf.keras.optimizers.AdamW(learning_rate=0.01),
    loss='sparse_categorical_crossentropy',  # Use sparse_categorical_crossentropy for integer labels
    metrics=['accuracy']
    
)


c:\Users\PUGAZH\Desktop\Projects\DeepLearning\FaceAge_Detection\dlvenv\lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [12]:

# Step 3: Set Up Callbacks
callbacks = [
    ModelCheckpoint(
        filepath='model_best.h5',  # Save the best model
        save_best_only=True,
        monitor='val_accuracy',  # Monitor validation accuracy
        mode='max',
        verbose=1
    ),
    EarlyStopping(
        patience=5,  # Stop training if no improvement for 5 epochs
        monitor='val_loss',
        mode='min',
        verbose=1,
        restore_best_weights=True
    ),
    ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.2,  # Reduce learning rate by a factor of 0.2
        patience=3,  # If no improvement for 3 epochs
        min_lr=1e-6,
        verbose=1
    )
]



In [14]:
# Step 4: Train the Model
history = model.fit(
    train_dataset,
    validation_data=test_dataset,
    epochs=10,
    
    callbacks=callbacks,
    verbose=1
)



Epoch 1/10
854/854 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.0852 - loss: 173.8300
Epoch 1: val_accuracy improved from -inf to 0.07668, saving model to model_best.h5


854/854 ━━━━━━━━━━━━━━━━━━━━ 1166s 1s/step - accuracy: 0.0852 - loss: 173.7273 - val_accuracy: 0.0767 - val_loss: 9.2749 - learning_rate: 0.0100
Epoch 2/10
691/854 ━━━━━━━━━━━━━━━━━━━━ 3:40 1s/step - accuracy: 0.0841 - loss: 6.8073

KeyboardInterrupt: 

In [15]:
# Step 5: Evaluate the Model
test_loss, test_accuracy = model.evaluate(test_dataset, verbose=1)
print(f"Test Accuracy: {test_accuracy:.2f}")

108/108 ━━━━━━━━━━━━━━━━━━━━ 28s 257ms/step - accuracy: 0.3198 - loss: 2.6868
Test Accuracy: 0.32


In [19]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.models import load_model

def predict_age_from_image(image_path, model_path='model_best.h5', image_size=(224, 224)):
    """
    Given an image file path, this function will:
    1. Load the pre-trained model from a specified path.
    2. Preprocess the image (resize and normalize).
    3. Make a prediction on the age.
    4. Return the predicted age.
    
    Parameters:
    - image_path: Path to the image file for which the age is to be predicted.
    - model_path: Path to the saved trained model (.h5 format).
    - image_size: Target size for resizing the image to match the model input (default: (224, 224)).
    
    Returns:
    - predicted_age: The predicted age (a discrete value).
    """
    
    try:
        # ------------------- Step 1: Load the Trained Model ------------------- #
        model = load_model(model_path)  # Load the model from the specified path
        print(f"Model loaded successfully from {model_path}")
        
    except Exception as e:
        print(f"Error loading the model: {e}")
        return None
    
    try:
        # ------------------- Step 2: Preprocess the Input Image ------------------- #
        # Load the image
        image = load_img(image_path, target_size=image_size)  # Resize the image
        image = img_to_array(image)  # Convert image to a NumPy array
        
        # Normalize the image (scale pixel values to range [0, 1])
        image = tf.cast(image, tf.float32) / 255.0
        
        # Add a batch dimension (model expects a batch of images, not just one)
        image = np.expand_dims(image, axis=0)  # Shape: (1, 224, 224, 3)
        print(f"Image preprocessed successfully: {image.shape}")
        
    except Exception as e:
        print(f"Error preprocessing the image {image_path}: {e}")
        return None
    
    try:
        # ------------------- Step 3: Make the Prediction ------------------- #
        # Predict the output using the model (returns a probability distribution over ages)
        prediction = model.predict(image)
        
        # Get the index of the highest probability (i.e., the predicted age)
        predicted_age = np.argmax(prediction)  # Index with the highest probability
        print(f"Predicted age: {predicted_age}")
        
        return predicted_age  # Return the predicted age
        
    except Exception as e:
        print(f"Error making prediction for {image_path}: {e}")
        return None

# Example usage:
image_path = 'path_to_image.jpg'  # Replace with the actual image path
predicted_age = predict_age_from_image(r"C:\Users\PUGAZH\Desktop\Projects\DeepLearning\FaceAge_Detection\src\Dataset\ProcessedData\81-90\81_0_0_20170111222146910.jpg.chip.jpg")

if predicted_age is not None:
    print(f"The predicted age for the given image is: {predicted_age}")


Model loaded successfully from model_best.h5
Image preprocessed successfully: (1, 224, 224, 3)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
Predicted age: 10
The predicted age for the given image is: 10
